In [2]:
import os
# import torch
import numpy as np
import pandas as pd
# from torch.utils.data import Dataset
import fitz
from tqdm import tqdm
import re

In [3]:
main_path = 'Дата-сет для задачи №1\\Разметка\\'

In [4]:
full_data = pd.read_csv('Дата-сет для задачи №1\\Дата-сет_Задача 1.csv', sep=';', encoding="windows-1251")

In [3]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100425 entries, 0 to 100424
Data columns (total 5 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Номер_в_госреестре              100416 non-null  object 
 1   Наименование_СИ                 99851 non-null   object 
 2   Единица_измерения_СИ            0 non-null       float64
 3   Погрешность_СИ                  0 non-null       float64
 4   Наименование_файла_с_описанием  51694 non-null   object 
dtypes: float64(2), object(3)
memory usage: 3.8+ MB


In [5]:
#

have_file_df = full_data[(full_data['Наименование_файла_с_описанием'].notna())]
print(have_file_df.shape)
test_files = os.listdir('Дата-сет для задачи №1\\Разметка')
print(len(test_files))
answer_df = full_data[full_data['Наименование_файла_с_описанием'].isin(test_files)]
print(answer_df.shape)
answer_df = answer_df.drop_duplicates(['Наименование_файла_с_описанием'])
print(answer_df.shape)

(51694, 5)
1106
(895, 5)
(886, 5)


In [6]:
file_names=[]
for i in range(answer_df.shape[0]):
    name = answer_df['Наименование_файла_с_описанием'].iloc[i]
    if os.path.exists(f'Дата-сет для задачи №1\\Разметка\\{name}'):
        file_names.append(f'Дата-сет для задачи №1\\Разметка\\{name}')

In [7]:
file_small_names = [name.split('\\')[-1] for name in file_names]
with open('file_small_names.txt', 'w') as f:
    for item in file_small_names:
        f.write("%s\n" % item)


### PDF HIGHLIGHTS READER

In [72]:
# new_data_dict = {}
# data_dict = {}
# for file_name in tqdm(file_names):
#     # list to store the co-ordinates of all highlights
#     highlights = []
#     # List to store all the highlighted texts
#     highlight_text = []
#     # read file
#     doc = fitz.open(file_name)
#     page_count = doc.page_count
#     for n in range(page_count):
#         # load page
#         page = doc.load_page(n)
#         # annot generator
#         annot = page.firstAnnot
#         # loop till we have highlight annotation in the page
#         while annot:
#             if annot.type[0] == 8:
#                 all_coordinates = annot.vertices
#                 if len(all_coordinates) == 4:
#                     highlight_coord = fitz.Quad(all_coordinates).rect
#                     highlights.append(highlight_coord)
#                 else:
#                     all_coordinates = [all_coordinates[x:x+4] for x in range(0, len(all_coordinates), 4)]
#                     for i in range(0,len(all_coordinates)):
#                         coord = fitz.Quad(all_coordinates[i]).rect
#                         highlights.append(coord)
#             annot = annot.next

#         # all words from page
#         all_words = page.get_text_words()
        
#         for h in highlights:
#             sentence = [w[4] for w in all_words if fitz.Rect(h).intersects(w[0:4])]
#             highlight_text.append(" ".join(sentence))    

#         data_dict[file_name] = highlight_text
#         new_data_dict[file_name.split('\\')[-1]] = highlight_text

100%|██████████| 886/886 [03:36<00:00,  4.08it/s]


In [64]:
def parse_pdf(file_name):
    '''
    Из всего пдф файла парсит highlights и возвращает текст и цвет
    '''
    # List to store all the highlighted texts
    highlight_text = []
    # read file
    doc = fitz.open(file_name)
    page_count = doc.page_count
    for n in range(page_count):
        # list to store the co-ordinates of all highlights
        highlights = []
        # load page
        page = doc.load_page(n)
        # annot generator
        annot = page.firstAnnot
        # loop till we have highlight annotation in the page
        while annot:
            if annot.type[0] == 8:
                all_coordinates = annot.vertices
                if len(all_coordinates) == 4:
                    highlight_coord = fitz.Quad(all_coordinates).rect
                    highlights.append((highlight_coord, annot.colors))

                else:
                    all_coordinates = [all_coordinates[x:x+4] for x in range(0, len(all_coordinates), 4)]
                    for i in range(0,len(all_coordinates)):
                        coord = fitz.Quad(all_coordinates[i]).rect
                        highlights.append((coord, annot.colors))
            annot = annot.next

        # all words from page
        all_words = page.get_text_words()
        highlights = sorted(highlights, key=lambda x: tuple(x[0])[1])
        for h in highlights:
            sentence = [w[4] for w in all_words if fitz.Rect(h[0]).intersects(w[0:4])]
            highlight_text.append((" ".join(sentence), h[1]))
        
    return highlight_text

In [78]:
def create_pdf_dict(highlight_text):
    def color_define(tup):
        if tup[2] > 0.9:
            return 'error'
        if tup[0] > 0.9:
            return 'name'
        else:
            return 'unit'

    my_dict = {'name': [], 'unit': [], 'error': []} # одна пдфка
    for i in highlight_text:
        my_dict[color_define(i[1]['stroke'])].append(i[0])
    for key in my_dict:
        my_dict[key] = ' '.join(my_dict[key])
    
    return my_dict


In [291]:
data = []
for file_name in tqdm(os.listdir(main_path)): #для обучения на пересекающейся выборке file_small_names
    tmp_dict = create_pdf_dict(parse_pdf(main_path+file_name))
    tmp_dict['file_name'] = file_name
    data.append(tmp_dict)
df = pd.DataFrame(data)

100%|██████████| 1106/1106 [02:07<00:00,  8.64it/s]


In [292]:
df['true_unit'] = df['unit'].str.extract(r"(, [°СА-я1-9]+)")
df.true_unit = df.true_unit.str.slice(2)

# Обработка Диапазон
str_diap_idx = df[df.true_unit.isin(['Диапазон'])].index
df.loc[str_diap_idx, 'unit'] = df.loc[str_diap_idx, 'unit'] + ' '
tmp_df = df[df.true_unit.isin(['Диапазон'])].unit.str.extract(r"(, [°СА-я•]{1,3}[1-9]? )")
df.loc[str_diap_idx, 'true_unit'] = tmp_df[0].str.slice(2,-1).values

# Обработка измерений, измерения
str_diap_idx = df[df.true_unit.isin(['измерений', 'измерения'])].index
df.loc[str_diap_idx, 'unit'] = df.loc[str_diap_idx, 'unit'] + ' '
tmp_df = df[df.true_unit.isin(['измерений', 'измерения'])].unit.str.extract(r"(, [°СА-я•]{1,5}[1-9]? )")
df.loc[str_diap_idx, 'true_unit'] = tmp_df[0].str.slice(2,-1).values

# Обработка Пределы
str_diap_idx = df[df.true_unit.isin(['Пределы'])].index
tmp_df = df[df.true_unit.isin(['Пределы'])].unit.str.extract(r"([°СА-я•]{1,3}[1-9]?)")
df.loc[str_diap_idx, 'true_unit'] = tmp_df[0].values

In [303]:
df.to_csv('unit_train_df.csv')

In [71]:
# Мой датафрейм
df = {i: list(item) for i, item in enumerate(data_dict.items())}
df = pd.DataFrame.from_dict(df, orient='index', columns=['file_name', 'text'])


In [73]:
# Общий датафрейм
df = {i: list(item) for i, item in enumerate(new_data_dict.items())}
df = pd.DataFrame.from_dict(df, orient='index', columns=['file_name', 'text'])
df.to_csv('train_df.csv')

### PDF Full texts for classification

In [300]:
data_dict = {}
for file_name in tqdm(os.listdir(main_path)):
        text=''
        doc = fitz.open(main_path + file_name)
        page_count = doc.page_count
        for n in range(page_count):
                page = doc.load_page(n)
                text += page.get_text()
        data_dict[file_name] = text


100%|██████████| 1106/1106 [01:07<00:00, 16.30it/s]


In [301]:
pdf_df = {i: list(item) for i, item in enumerate(data_dict.items())}
pdf_df = pd.DataFrame.from_dict(pdf_df, orient='index', columns=['file_name', 'text'])
pdf_df.to_csv('full_train_df.csv')

In [304]:
pdf_df

,file_name,text
0,2005-30815-05.pdf,Трансформатор напряжения эталонный\nмобильный ...
1,2008-01054-08.pdf,
2,2008-27361-04.pdf,Счетчики скоростные крыльчатые\nхолодной и гор...
3,2008-34048-08.pdf,Счетчики активной\nэлектрической энергии трехф...
4,2008-34291-07.pdf,Внесены в Государственный реестр\nсредств изме...
...,...,...
1101,2021-82991-21.pdf,УТВЕРЖДЕНО \nприказом Федерального агентства \...
1102,2021-83010-21.pdf,УТВЕРЖДЕНО \nприказом Федерального агентства \...
1103,2021-83333-21.pdf,\nУТВЕРЖДЕНО \nприказом Федерального агентств...
1104,2021-83360-21.pdf,\nУТВЕРЖДЕНО \nприказом Федерального агентств...
